In [ ]:
#hide
from timeseries.core import *

# Welcome to timeseries for fastai v2
> timeseries is an unofficial extension package for fastai v2. It implements timeseries classification using Neural Networks models
> such as InceptionTime

This file will become your README and also the index of your documentation.

## Install

`pip install timeseries -c ai-fast-track`

## How to use

timeseries follows the same 

In [ ]:
1+1

2

## Tests

To run the tests in parallel, launch:

```bash
nbdev_test_nbs
```
or 
```bash
make test
```

## Contributing


After you clone this repository, please run nbdev_install_git_hooks in your terminal. This sets up git hooks, which clean up the notebooks to remove the extraneous stuff stored in the notebooks (e.g. which cells you ran) which causes unnecessary merge conflicts.

Before submitting a PR, check that the local library and notebooks match. The script nbdev_diff_nbs can let you know if there is a difference between the local library and the notebooks.

If you made a change to the notebooks in one of the exported cells, you can export it to the library with nbdev_build_lib or make fastai2.
If you made a change to the library, you can export it back to the notebooks with nbdev_update_lib.